# Linear Regression

In this tutorial we will implement a linear regression model. We will also implement a function that splits the available data into a training and a testing part.

## Problem Setting

We will use the Boston Housing Dataset. This dataset contains information collected by the U.S Census Service concerning housing in the city of Boston in the state of Massachusetts in 1978. Our goal is to predict the median value of the houses in a particular town in the city of Boston given its attributes. Check the file ’housing_features_description.txt’ for more information on the attributes.

In [ ]:
import urllib
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

### NOTE: The boston dataset has an ethical problem. More on this can e.g. be found in the scikit documentation. ###
### Summary: The dataset contains the proportion of black people, which was assumed that racial self-segregation had a positive impact on house prices. ###


data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
boston_data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
boston_target = raw_df.values[1::2, 2]

df=pd.DataFrame(boston_data)
df.columns=['crime_rate','res_land_zoned','industry','charles_river','nox','avg_num_rooms','prop_bf_1940','dst_emply_center','rd_highway_idx','tax_rate','stdnt_tchr_ratio','prop_blacks','low_status_pct']
X=boston_data
y=boston_target

# Our goal is to predict the median value of the houses in a particular town in the city of Boston given its attributes. 

<>:14: SyntaxWarning: invalid escape sequence '\s'
<>:14: SyntaxWarning: invalid escape sequence '\s'
C:\Users\juand\AppData\Local\Temp\ipykernel_3092\4038728560.py:14: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,crime_rate,res_land_zoned,industry,charles_river,nox,avg_num_rooms,prop_bf_1940,dst_emply_center,rd_highway_idx,tax_rate,stdnt_tchr_ratio,prop_blacks,low_status_pct
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [ ]:
df.head(10)
summary = pd.DataFrame({
    'mean': df.mean(),
    'std':  df.std(),
    'median': df.median()
}).round(3)
print(summary)
df.info()

# We need to make sure all our data is there, and also check any meaningful information (what ranges are we working with and such)

#  Variables in order:
#  CRIM     per capita crime rate by town
#  ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
#  INDUS    proportion of non-retail business acres per town
#  CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
#  NOX      nitric oxides concentration (parts per 10 million)
#  RM       average number of rooms per dwelling
#  AGE      proportion of owner-occupied units built prior to 1940
#  DIS      weighted distances to five Boston employment centres
#  RAD      index of accessibility to radial highways
#  TAX      full-value property-tax rate per $10,000
#  PTRATIO  pupil-teacher ratio by town
#  B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
#  LSTAT    % lower status of the population
#  MEDV     Median value of owner-occupied homes in $1000's

                     mean      std   median
crime_rate          3.614    8.602    0.257
res_land_zoned     11.364   23.322    0.000
industry           11.137    6.860    9.690
charles_river       0.069    0.254    0.000
nox                 0.555    0.116    0.538
avg_num_rooms       6.285    0.703    6.208
prop_bf_1940       68.575   28.149   77.500
dst_emply_center    3.795    2.106    3.207
rd_highway_idx      9.549    8.707    5.000
tax_rate          408.237  168.537  330.000
stdnt_tchr_ratio   18.456    2.165   19.050
prop_blacks       356.674   91.295  391.440
low_status_pct     12.653    7.141   11.360
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   crime_rate        506 non-null    float64
 1   res_land_zoned    506 non-null    float64
 2   industry          506 non-null    float64
 3   charles_river     506 non-null    float

# Note 1:

Think about the ethical aspects of this dataset and machine learning in general. 

Can you always trust your data source? Can we use every possible information for our models?



In [ ]:
# Data sources will always have some kind of bias, especially when it comes to social sciences. 
# Thinking about how data is obtained and by who is always important to understand the inherent biases and also avoid falling in the same logic flaws.

### Exercise 1

Write the *split_train_test(X,y,split,seed)*, given an instance matrix $X \in \mathbb{R}^{N \times D}$, labels $y \in Y^N$, a split ratio in $[0, 1]$ and a random seed $\in \mathbb{Z}$. Split the dataset in $(split×100)\%$ of the instances for training our model and the rest for testing, i.e. 

$$ \left|X_{\text{train}}\right| = \lceil \text{split} \cdot N \rceil, \qquad |X_{\text{train}}| + |X_{\text{test}}| = N. $$
Make sure you use the given random number generator seed so we all get the same results. The function is supposed to return:

- X_train, y_train: the training instances and labels;
- X_test, y_test: the test instances and labels,

in the same order as was mentioned.

Hint: It may be helpful to use shuffling functionality (e.g. np.random.shuffle).

In [ ]:


def split_train_test(X,y,split,seed):
    ##################
    np.random.seed(seed)
    
    N = len(X)
    index = np.arange(N)
    np.random.shuffle(index)
    
    train_examples = int((split / 100) * N)
    test_examples = N - train_examples
    
    train_index = index[:train_examples]
    test_index = index[-test_examples:]
    
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index],y[test_index]
    print(len(X_train))
    print(len(X_test), test_examples)
    #INSERT CODE HERE#
    ##################
    return X_train, y_train, X_test, y_test



101
405 405
hi


### Exercise 2

Write the function *train_linear_reg(X_train,y_train,lmbd)*.
Implement the ridge regression model (slide 24). The function should output the learned weight vector $\theta \in \mathbb{R}^D$ or $\mathbb{R}^{D+1}$ (depending on whether you are adding *bias*).

In [ ]:
def train_linear_reg(X, y, lmbd):
    
    
    ##################
    theta = np.dot(np.linalg.inv(np.dot(X.T, X) + (lmbd * np.eye(X.shape[1]))), np.dot(X.T, y))
    ##################
    return theta

# To add a bias



# Try to actually remember what's the deal with ridge regression.
# So ridge regression is a combination of Squared loss + L2 regularization.
# It's a convex optimization criterion, with only one global minimum.

# Only practical for relatively small number of attributes.
# Otherwise: use stochastic gradient method. 
# Why? Ridge regression has a heavy memory cost because of inverting matrixes. 
# SGD however uses small batches for trainig, so all good.



### Exercise 3

Write the function *predict(X,theta)* which predicts housing values vector pred for a dataset X and a previously trained parameter vector $\theta$.

In [ ]:
def predict(X, theta):
    ##################
    y_pred = np.dot(X, theta)
    ##################
    return y_pred

### Exercise 4

Write the function *mean_abs_loss(y_true,y_pred)* which computes the mean of the absolute differences between our prediction vector $y\_pred$ and the real housing values $y\_true$.

In [ ]:
def mean_abs_loss(y_true,y_pred):
    ##################
    #INSERT CODE HERE#
    ##################
    return 0

### Exercise 5

Evaluate your solutions by running the following code. 

Moreover, answer the following questions: What is the most important feature in your model? Are there features that are not so important? What happens if you remove them? Are there outliers with a high absolute loss?

In [ ]:
seed = 3
lmbd=1
split=0.7
X_train,y_train,X_test,y_test=split_train_test(X,y,split,seed)
theta=train_linear_reg(X_train,y_train,lmbd)
y_pred=predict(X_test,theta)
mae=mean_abs_loss(y_test,y_pred)
print ('The mean absolute loss is {loss:0.3f}'.format(loss=mae*1000))